In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.linear_model import LassoCV, RidgeCV
import statsmodels.api as sm

In [2]:
def root_mean_log_squared_error(y_test, predicted):
    return np.sqrt(np.sum((np.log(1+y_test)-np.log(1+predicted))**2)/len(predicted))

In [3]:
data = pd.read_csv('train.csv')
data_test = pd.read_csv('test.csv')

# Обработка данных

In [4]:
answer = pd.DataFrame()
answer['Id'] = data_test['Id']

In [ ]:
y = data['SalePrice']
del data['SalePrice'] 

In [ ]:
#объединение сетов
data_full = pd.concat([data,data_test])

In [ ]:
# Percentage of missing values
mis_val_percent = (100 * data_full.isnull().sum() /len(data_full)).sort_values(ascending=False)[:10]
mis_val_percent

In [ ]:
#удаление столбцов с NaN>80
data_full = data_full.drop(['Alley','PoolQC','MiscFeature','Fence'], axis = 1)

In [ ]:
#Заполнение категориальных NaN
for col in ('FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'BsmtFinType2', 
            'BsmtExposure', 'BsmtFinType1', 'BsmtCond', 'BsmtQual', 'MasVnrType', 'MSZoning', 'Functional', 'Utilities', 
            'SaleType', 'KitchenQual', 'Exterior1st','Exterior2nd','Electrical' ):
    data_full[col] = data_full[col].fillna('None')

In [ ]:
#Заполнение числовых NaN 
for col in ['GarageYrBlt', 'GarageArea', 'GarageCars', 'MasVnrArea', 'BsmtHalfBath', 'BsmtFullBath', 'TotalBsmtSF', 'BsmtUnfSF',
           'BsmtFinSF2', 'BsmtFinSF1','LotFrontage']:
    data_full[col] = data_full[col].fillna(int(0))

In [ ]:
#Корреляция
#data_full = data_full.drop(['Id', 'BsmtFinSF2', 'BsmtUnfSF', 'BsmtHalfBath','LowQualFinSF',
      #        'GarageArea', 'MoSold', 'MiscVal', 'OpenPorchSF', 'YrSold', 'PoolArea', 
   # #          '3SsnPorch', 'EnclosedPorch', 'HalfBath', 'GrLivArea', 'FullBath'], axis = 1)

In [ ]:
#Кодирование
data_full = pd.get_dummies(data_full)
data_test = data_full.iloc[data.shape[0]:,:]
data = data_full.iloc[:data.shape[0],:]

In [ ]:
print(np.all(data.columns==data_test.columns)) 

True


# Построение модели

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.2, random_state=22)

In [ ]:
#установка максимальной степени
poly_reg = PolynomialFeatures(degree = 2)

In [ ]:
#возведение данных в степень
X_poly_train = poly_reg.fit_transform(X_train)
X_poly_test = poly_reg.transform(X_test)
data_test_poly = poly_reg.transform(data_test)

In [ ]:
#наполнение линейной модели данными
regressor = LinearRegression()
regressor.fit(X_poly_train, y_train)

In [ ]:
root_mean_log_squared_error(y_test, regressor.predict(X_poly_test))